In [1]:
%load_ext autoreload
%load_ext memory_profiler 
%load_ext dotenv
%autoreload 2
%dotenv

In [2]:
import torch
import logging
from lightning import Trainer
from geovision.experiment.config import ExperimentConfig
from geovision.models.interfaces import ClassificationModule
from geovision.data import ImageDatasetDataModule

from geovision.io.local import FileSystemIO as fs
from geovision.experiment.loggers import (
    get_csv_logger, 
    get_ckpt_logger,
    get_classification_logger
)

In [3]:
config = ExperimentConfig.from_yaml("config.yaml")
datamodule = ImageDatasetDataModule(config.dataset_constructor, config.dataset_config, config.dataloader_config)

logging.basicConfig(
    filename = f"{fs.get_new_dir(config.experiments_dir, "logs")}/logfile.log",
    filemode = "a",
    format = "%(asctime)s : %(name)s : %(levelname)s : %(message)s",
    level = logging.INFO
)

loggers: list = list()
loggers.append(csv_logger := get_csv_logger(config))

callbacks: list = list()
callbacks.append(ckpt_logger := get_ckpt_logger(config))
callbacks.append(metrics_logger := get_classification_logger(config))

In [4]:
config = ExperimentConfig.from_yaml("config.yaml")
model = ClassificationModule(
    model_config=config.model_config,
    criterion_constructor=config.criterion_constructor, 
    criterion_params=config.criterion_params, 
    optimizer_constructor=config.optimizer_constructor,
    optimizer_params=config.optimizer_params,
    lr_scheduler_constructor=config.scheduler_constructor,
    lr_scheduler_params=config.scheduler_params,
    warmup_scheduler_constructor=config.warmup_scheduler_constructor,
    warmup_scheduler_params=config.warmup_scheduler_params,
    warmup_steps=config.warmup_steps,
    scheduler_config_params=config.scheduler_config_params
)
# model = torch.compile(model)
trainer = Trainer(logger = loggers, callbacks = callbacks, **config.trainer_params)
trainer.fit(
    model = model, 
    datamodule = datamodule, 
    ckpt_path = config.ckpt_path 
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/sambhav/miniconda3/envs/dev/lib/python3.12/site-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory /home/sambhav/experiments/inria/testing_everything exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!


Empty DataFrame
Columns: [ckpt_path, epoch, step, version]
Index: []


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                   | Params | Mode 
-------------------------------------------------------------
0 | encoder   | ResNetFeatureExtractor | 11.2 M | train
1 | decoder   | UNetDecoder            | 7.2 M  | train
2 | criterion | CrossEntropyLoss       | 0      | train
-------------------------------------------------------------
18.4 M    Trainable params
0         Non-trainable params
18.4 M    Total params
73.410    Total estimated model params size (MB)
182       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

BackendCompilerFailed: backend='inductor' raised:
RuntimeError: Found NVIDIA GeForce GTX 1060 3GB which is too old to be supported by the triton GPU compiler, which is used as the backend. Triton only supports devices of CUDA Capability >= 7.0, but your device is of CUDA capability 6.1

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True
